In [1]:
import numpy as np
import matplotlib.pyplot as plt
from tqdm import tqdm
from separateChannel3DNet import separateChannel3DNet
from twoChannel3DNet import twoChannel3DNet
from colorMNist import colorMNist
import torch
import pickle

In [7]:
def avg_acc(x, y, mtype, dataset_length, n_exp, model_layers1, model_layers2, features, mix, hue_shift=-1):
    accuracies = []
    for iteration in tqdm(range(2, 13, 2), total=6, desc=f"{x} {y} {mtype} {dataset_length}"):
        # Load dataset
        with open("custom_datasets/" + str(dataset_length) + '/' + y + ".pkl", "rb") as f:
            cmnist_train, cmnist_val, cmnist_test = pickle.load(f)
            test_dataset = colorMNist(cmnist_test)
            test_dataloader = torch.utils.data.DataLoader(test_dataset, batch_size=32, shuffle=False, num_workers = 0)

            # Layers of the model
            model_layers = model_layers1 if mtype == 1 else model_layers2
            # Create model
            model = separateChannel3DNet(model_layers, model_layers[-2], features, mix) if mtype == 1 else twoChannel3DNet(model_layers, model_layers[-2], features, mix)
            # Load file
            extra = "" if hue_shift == -1 else "_" + str(hue_shift)
            lfile = str(mtype) + "-Gaussian3D_" + x + extra
            # Load model
            model.load_state_dict(torch.load('Experiments/conv/model_saves/' + str(n_exp) + '/' + str(dataset_length)
                                             + '/' + str(iteration) + '/' + lfile + '.pth'))
            # Put model on gpu
            model.cuda()

            # Total and amount correct
            test_correct = 0
            test_total = 0

            # Put the model in evaluation mode
            model.eval()

            # Without gradient calculation
            with torch.no_grad():
                for (images, labels) in test_dataloader:

                    # Add extra dimension for the network
                    images = images.unsqueeze(1)

                    # Put images
                    images = images.cuda()

                    # Predicted labels
                    preds = model(images)

                    # Top predictions per image
                    _, top_preds = torch.max(preds, 1)

                    # Predictions and images back on cpu
                    top_preds = top_preds.cpu()
                    images = images.cpu()

                    # Amount of correct predictions
                    predictions = [top_preds[i].item() == labels[i].item() for i in range(len(labels))]
                    correct = np.sum(predictions)

                    # Update total correct and total images
                    test_correct += correct
                    test_total += len(images)

            accuracies.append(test_correct / test_total)
    return accuracies

In [10]:
datasets = [["0", "cmnist_deterministic"], ["12", "cmnist_gaussian_12"], ["60", "cmnist_gaussian_60"],
            ["120", "cmnist_gaussian_120"], ["uniform", "cmnist_gaussian_uniform"]]
dataset_lengths = ["1k", "2k", "5k", "10k", "20k", "60k"]
n_exp = 2
model_layers1 = [32, "M", 64, "M", 128, "M"]
model_layers2 = [32, "M", 64, "M", 128, "M"]
features = [3, 1, 1]
mix = True

for dl in dataset_lengths:
    tot_accs1 = []
    tot_accs2 = []
    for x, y in datasets:
        tot_accs1.append(avg_acc(x, y, 1, dl, n_exp, model_layers1, model_layers2, features, mix))
        tot_accs2.append(avg_acc(x, y, 2, dl, n_exp, model_layers2, model_layers2, features, mix))

    y1_mean = []
    y1_std = []
    y2_mean = []
    y2_std = []
    for i in range(len(tot_accs1)):
        y1_mean.append(np.mean(tot_accs1[i]))
        y1_std.append(np.std(tot_accs1[i]))
        y2_mean.append(np.mean(tot_accs2[i]))
        y2_std.append(np.std(tot_accs2[i]))

    np.savetxt("Experiments/conv/plots/" + str(n_exp) + "/" + dl + "_1cv2c.txt", [y1_mean, y1_std, y2_mean, y2_std])

uniform cmnist_gaussian_uniform 2 60k: 100%|██████████| 6/6 [00:35<00:00,  5.85s/it]
